### Imports

In [13]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from astropy.io import fits
%config InlineBackend.rc = {}

In [14]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

### Image Wrangling

These are to be used in the case that I transform my FITS image files into JPGs.

In [17]:
hdu_list = fits.open('efigi-images/psf_i/psf_PGC0000212_i.fits')
hdu_list.info()

Filename: efigi-images/psf_i/psf_PGC0000212_i.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  chip01        1 PrimaryHDU      19   (45, 45)   float32   


This tells us there are no ASCII tables containing data within the FITS headers

In [19]:
# Image Data Generator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [7]:
# Train and Test Data

train_generator = train_datagen.flow_from_directory('images/train',
                                                    target_size=(,),
                                                    class_mode='categorical'
                                                   )

test_generator = test_datagen.flow_from_directory('images_test',
                                                  target_size=(,),
                                                  class_mode='categorical'
                                                 )


In [10]:
# Define Filters

### Modeling

#### First Simple Model

In [20]:
# Model Build

fsm_model = Sequential()
fsm_model.add(ConvD(64, (4,4), input_shape=(,,), activation='relu'))
fsm_model.add(Flatten())
fsm_model.add(Dense(units=64, activation='relu'))
fsm_model.add(Dense(units=18, activation='softmax'))

In [ ]:
# Fitting Model

fsm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

fsm_model.fit(x_train, y_train,
         epochs=10,
         verbose=0,
         validation_data=(x_test, y_test))

fsm_score = fsm_model.evaluate(x_test, y_test)

In [ ]:
# Evaluation

Notes: 

#### Model with additional Pooling Layers

In [ ]:
# Model Build

pool_model = Sequential()
pool_model.add(Conv2D(32, (4,4), input_shape=(,,), activation='relu'))
pool_model.add(Conv2D(64, (4,4), activation='relu'))
pool_model.add(MaxPooling2D)
pool_model.add(Conv2D(128))
pool_model.add(MaxPooling2D)
pool_model.add(Dense(18, activation='softmax'))

In [ ]:
# Fitting Model

pool_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

pool_model.fit(x_train, y_train,
         epochs=10,
         verbose=0,
         validation_data=(x_test, y_test))

pool_score = pool_model.evaluate(x_test, y_test)

In [ ]:
# Evaluation

Notes: 

### Model with Dropout

This project values on robust capabilities in handling edge cases, so we want to take extra care in protecting against overfitting. This iteration adds in two dropout layers, the first significant and the second less imposing. 

In [8]:
# Model Build

model = Sequential()
model.add(Conv2D(32, (4,4), input_shape=(,,), activation='relu'))
model.add(Conv2D(64, (4,4), activation='relu'))
model.add(MaxPooling2D)
model.add(Conv2D(128))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(18, activation='softmax'))

In [9]:
# Fitting Model

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.fit()

In [ ]:
# Evaluation